In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
from transformers import RobertaModel, RobertaTokenizer
import torch

In [4]:
file_path = '/content/drive/MyDrive/free_topic /text/text_datasetADDRESSO.csv'
df = pd.read_csv(file_path, sep=',')
#print(df.iloc[0:10])
print(df.head())
df

                   file_name  \
0  adrso024trimmed_plain.txt   
1  adrso028trimmed_plain.txt   
2  adrso033trimmed_plain.txt   
3  adrso035trimmed_plain.txt   
4  adrso036trimmed_plain.txt   

                                                text  labels  
0  The cookie jar and the little boy about down o...       1  
1  dishes and Stu is upset you know she's getting...       1  
2  in here this way i really don't know because i...       1  
3  Oh boy. Well; the little boy is reaching for a...       1  
4  how the boy is up behind the cage and the woma...       1  


,file_name,text,labels
0,adrso024trimmed_plain.txt,The cookie jar and the little boy about down o...,1
1,adrso028trimmed_plain.txt,dishes and Stu is upset you know she's getting...,1
2,adrso033trimmed_plain.txt,in here this way i really don't know because i...,1
3,adrso035trimmed_plain.txt,Oh boy. Well; the little boy is reaching for a...,1
4,adrso036trimmed_plain.txt,how the boy is up behind the cage and the woma...,1
...,...,...,...
132,adrso309_plain.txt,Mother is drying the dishes; looking out the w...,0
133,adrso310_plain.txt,Well; the water's running over. The boy's goin...,0
134,adrso312_plain.txt,Oh; I see the sink that's running over. I see ...,0
135,adrso315_plain.txt,The girl is reaching for a cookie that the boy...,0


In [5]:
def get_roberta_embeddings(texts, use_cls_token=False):
    # Initialize tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaModel.from_pretrained('roberta-base')

    # Tokenize input texts
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=512)

    # Forward pass, get hidden states
    with torch.no_grad():
        outputs = model(**encodings)

    # HOW TO POOL EMBEDDINGS: https://blog.ml6.eu/the-art-of-pooling-embeddings-c56575114cf8
    if use_cls_token:
        # Use the embedding of the CLS token (first token)
        embeddings = outputs.last_hidden_state[:, 0]
    else:
        # # Average the embeddings of all tokens
        # embeddings = outputs.last_hidden_state.mean(dim=1)
        attention_mask = encodings['attention_mask']
        masked_output = outputs.last_hidden_state * attention_mask.unsqueeze(-1)
        embeddings = masked_output.sum(dim=1) / attention_mask.sum(-1, keepdim=True)

    # Calculate the L2 norm of each embedding vector
    # norms = torch.norm(embeddings, p=2, dim=1).numpy()  # L2 norm along the feature dimension
    return embeddings

In [6]:
embeddings = get_roberta_embeddings(df['text'].tolist(), use_cls_token=False)
#print(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df['embeddings'] = get_roberta_embeddings(df['text'].tolist(), use_cls_token=True).tolist()
print(df.head())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                   file_name  \
0  adrso024trimmed_plain.txt   
1  adrso028trimmed_plain.txt   
2  adrso033trimmed_plain.txt   
3  adrso035trimmed_plain.txt   
4  adrso036trimmed_plain.txt   

                                                text  labels  \
0  The cookie jar and the little boy about down o...       1   
1  dishes and Stu is upset you know she's getting...       1   
2  in here this way i really don't know because i...       1   
3  Oh boy. Well; the little boy is reaching for a...       1   
4  how the boy is up behind the cage and the woma...       1   

                                          embeddings  
0  [-0.09942031651735306, 0.0756431296467781, -0....  
1  [-0.08732760697603226, 0.08142437040805817, -0...  
2  [-0.048910606652498245, 0.0464143268764019, -0...  
3  [-0.09717251360416412, 0.09806136041879654, -0...  
4  [-0.13010969758033752, 0.06933900713920593, -0...  


In [ ]:
#df.to_csv('/content/frozen_embeddings.csv', index=False)